In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

#Seed the notebool
import random
random.seed(113)

In [ ]:
df = pd.read_csv('../input/train.csv',nrows =2000000 )
test_set = pd.read_csv("../input/test.csv")

In [ ]:
df = df.round(2)
df = df.reindex(np.random.permutation(df.index))


In [ ]:
def distance(lat1, lon1, lat2, lon2):
    distance = np.abs(lat2 - lat1) + np.abs(lon2 - lon1)
    return distance
'''    p = 0.017453292519943295 # Pi/180
    a = 0.5 - np.cos((lat2 - lat1) * p)/2 + np.cos(lat1 * p) * np.cos(lat2 * p) * (1 - np.cos((lon2 - lon1) * p)) / 2
    return 12742 * np.arcsin(np.sqrt(a)) # 2*R*asin...'''
#features from basic linear model kernel
def add_travel_vector_features(df):
    df['abs_diff_longitude'] = (df.dropoff_longitude - df.pickup_longitude).abs()
    df['abs_diff_latitude'] = (df.dropoff_latitude - df.pickup_latitude).abs()
    

jfk=(-73.7900, 40.6437)
lgr=(-73.8733, 40.7746)
ewr=(-74.1843, 40.6924)

# add new column to dataframe with distance in km
df['distance_km'] = distance(df.pickup_latitude, df.pickup_longitude, \
                                   df.dropoff_latitude, df.dropoff_longitude)

test_set['distance_km'] = distance(test_set.pickup_latitude, test_set.pickup_longitude, \
                                   test_set.dropoff_latitude, test_set.dropoff_longitude)
#JFK
# add new column to dataframe with distance in km
df['jfkpickup_distance_km'] = distance(df.pickup_latitude, df.pickup_longitude, \
                                   jfk[1], jfk[0])

test_set['jfkpickup_distance_km'] = distance(test_set.pickup_latitude, test_set.pickup_longitude, \
                                   jfk[1], jfk[0])

#La Guardia
# add new column to dataframe with distance in km
df['lgrpickup_distance_km'] = distance(df.pickup_latitude, df.pickup_longitude, \
                                   lgr[1], lgr[0])

test_set['lgrpickup_distance_km'] = distance(test_set.pickup_latitude, test_set.pickup_longitude, \
                                   lgr[1], lgr[0])

#La Guardia
# add new column to dataframe with distance in km
df['ewrpickup_distance_km'] = distance(df.pickup_latitude, df.pickup_longitude, \
                                   ewr[1], ewr[0])

test_set['ewrpickup_distance_km'] = distance(test_set.pickup_latitude, test_set.pickup_longitude, \
                                   ewr[1], ewr[0])
#DRop off distance
# add new column to dataframe with distance in km
df['jfkdropoff_distance_km'] = distance(df.dropoff_latitude, df.dropoff_longitude, \
                                   jfk[1], jfk[0])

test_set['jfkdropoff_distance_km'] = distance(test_set.dropoff_latitude, test_set.dropoff_longitude, \
                                   jfk[1], jfk[0])

#DRop off distance
# add new column to dataframe with distance in km
df['lgrdropoff_distance_km'] = distance(df.dropoff_latitude, df.dropoff_longitude, \
                                   lgr[1], lgr[0])

test_set['lgrdropoff_distance_km'] = distance(test_set.dropoff_latitude, test_set.dropoff_longitude, \
                                   lgr[1], lgr[0])

#DRop off distance
# add new column to dataframe with distance in km
df['ewrdropoff_distance_km'] = distance(df.dropoff_latitude, df.dropoff_longitude, \
                                   ewr[1], ewr[0])

test_set['ewrdropoff_distance_km'] = distance(test_set.dropoff_latitude, test_set.dropoff_longitude, \
                                   ewr[1], ewr[0])
add_travel_vector_features(df)
add_travel_vector_features(test_set)

In [ ]:
BB = (-75, -73, 40, 41.5)

# this function will be used with the test set below
def select_within_boundingbox(df, BB):
    return (df.pickup_longitude >= BB[0]) & (df.pickup_longitude <= BB[1]) & \
           (df.pickup_latitude >= BB[2]) & (df.pickup_latitude <= BB[3]) & \
           (df.dropoff_longitude >= BB[0]) & (df.dropoff_longitude <= BB[1]) & \
           (df.dropoff_latitude >= BB[2]) & (df.dropoff_latitude <= BB[3])
           

print('Old size: %d' % len(df))
df = df[select_within_boundingbox(df, BB)]
df = df[(df.passenger_count > 0) & (df.passenger_count <= 8)]
df = df[(df.fare_amount>0) & (df.fare_amount<=250)]
#test_set = test_set[select_within_boundingbox(test_set, BB)]
print('New size: %d' % len(df))

In [ ]:
def add_datetime_info(dataset):
    #Convert to datetime format
    dataset['pickup_datetime'] = pd.to_datetime(dataset['pickup_datetime'])
    
    dataset['hour'] = dataset.pickup_datetime.dt.hour
    dataset['day'] = dataset.pickup_datetime.dt.day
    dataset['month'] = dataset.pickup_datetime.dt.month
    dataset['weekday'] = dataset.pickup_datetime.dt.weekday
    dataset['year'] = dataset.pickup_datetime.dt.year
    
    return dataset

df = add_datetime_info(df)
test_set = add_datetime_info(test_set)


In [ ]:
df['year'].hist()


In [ ]:
test_set['year'].hist()

In [ ]:
airport_long = [-73.79,-73.87,-74.18]
airport_lat = [40.64,40.77,40.69]
print(df['dropoff_longitude'].describe())
'''jfk=(-73.79, 40.64)
lgr=(-73.87, 40.77)
ewr=(-74.18, 40.69)
airportList=[jfk,lgr,ewr]
def checkAirport(df):
    val = 0
    if (df['dropoff_longitude'] in airport_long) and (df['dropoff_latitude'] in airport_lat):
        val= 1
    else:
        val = 0
    return val
df['is_airport'] = df.apply(checkAirport,axis=1) 
df['is_airport'].value_counts()  '''        

In [ ]:
#df['is_night'] = np.where((((df['hour']>=20) & (df['hour']<=23)) | ((df['hour']>=0) & (df['hour']<6))),1,0) 
#df['is_airport'] = np.where((((df['dropoff_longitude']>=73.77) & (df['dropoff_longitude']<=73.78)) | ((df['dropoff_latitude']>=40.63) & (df['dropoff_latitude']<=40.64))),1,0) 
#df['is_surge']= np.where((((df['hour']>=16) & (df['hour']<20)) & ((df['weekday']!=5) & (df['weekday']!=6))),1,0)
#df['is_BeforeRaise']= np.where(((df['year']>=2012) & (df['month']>=8)),0,1)

In [ ]:
cor_frame = df.corr()
cor_frame.sort_values('fare_amount',axis=0)

In [ ]:
from sklearn.model_selection import train_test_split

train = df.drop(['key','fare_amount','pickup_datetime'],axis=1)#'hour','day','month','weekday','lgrpickup_distance_km',
test = df['fare_amount']
X_train, X_test, y_train, y_test = train_test_split(train, test, test_size = 0.2)

In [ ]:
'''from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_transformed = scaler.transform(X_train)
X_test_transformed = scaler.transform(X_test)'''


In [ ]:
'''from xgboost import XGBRegressor
regressor = XGBRegressor(max_depth =40, learning_rate=0.20, n_estimators=200,early_stopping_rounds=20)
regressor.fit(X_train, y_train)'''

In [ ]:
import xgboost as xgb
from bayes_opt import BayesianOptimization

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test)
params = {'eval_metric': 'rmse',
              'max_depth': 30,
              'subsample': 0.8,
              'eta': 0.1,
              'gamma': 1,
              'colsample_bytree': 1}
def xgb_evaluate(max_depth, gamma, colsample_bytree):
    
    # Used around 1000 boosting rounds in the full model
    cv_result = xgb.cv(params, dtrain, num_boost_round=100, nfold=3)    
    
    # Bayesian optimization only knows how to maximize, not minimize, so return the negative RMSE
    return -1.0 * cv_result['test-rmse-mean'].iloc[-1]
'''xgb_bo = BayesianOptimization(xgb_evaluate, {'max_depth': (3, 7), 
                                             'gamma': (0, 1),
                                             'colsample_bytree': (0.3, 0.9)})
# Use the expected improvement acquisition function to handle negative numbers
# Optimally needs quite a few more initiation points and number of iterations
xgb_bo.maximize(init_points=3, n_iter=5, acq='ei')

params = xgb_bo.res['max']['max_params']
params['max_depth'] = int(params['max_depth'])'''
# Train a new model with the best parameters from the search
model2 = xgb.train(params, dtrain, num_boost_round=200)



In [ ]:
import pickle 
#pickle.dump(regressor, open("xgbregressor", 'wb'))
 
# some time later...
 
# load the model from disk
#regressor = pickle.load(open("xgbregressor", 'rb'))

In [ ]:
predictions = model2.predict(dtest)
predictions_train = model2.predict(dtrain)

In [ ]:
import math
from sklearn.metrics import mean_squared_error

rmse = math.sqrt(mean_squared_error(y_test,predictions))
rmse_train = math.sqrt(mean_squared_error(y_train,predictions_train))

print(rmse)
print(rmse_train)


In [ ]:
'''from keras import Sequential
from keras import layers
from keras.layers import Dense,Dropout
from keras.callbacks import ModelCheckpoint,  ReduceLROnPlateau
from keras import optimizers
from keras.regularizers import l2

neuralNetwork = Sequential()
neuralNetwork.add(Dense(units=2056,input_shape=(14,),activation='relu',kernel_regularizer = l2(1e-2)))
neuralNetwork.add(Dropout(0.5))
neuralNetwork.add(Dense(units=10,activation='relu',kernel_regularizer = l2(1e-2)))
neuralNetwork.add(Dropout(0.5))
neuralNetwork.add(Dense(units=1,activation='relu',kernel_regularizer = l2(1e-2)))

neuralNetwork.compile(optimizer='Adam', 
           loss='mean_squared_error')

filepath = './model_weights/weights-improvement-10M.hdf5'
best_callback = ModelCheckpoint(filepath, 
                                save_best_only=True)

#lr_sched = ReduceLROnPlateau(monitor='loss', factor = 0.2, patience = 10, verbose = 1)

history = neuralNetwork.fit(X_train1, y_train, 
          epochs=20,
          verbose=0,
          batch_size=2048)

y_pred = neuralNetwork.predict(X_test1)
rmse = math.sqrt(mean_squared_error(y_test,y_pred))

print(rmse)'''

In [ ]:

#test_set['is_night'] = np.where((((test_set['hour']>=20) & (test_set['hour']<=23)) | ((test_set['hour']>=0) & (test_set['hour']<6))),1,0) 
#test_set['is_airport'] = np.where((((test_set['dropoff_longitude']>=73.77) & (test_set['dropoff_longitude']<=73.78)) | ((test_set['dropoff_latitude']>=40.63) & (test_set['dropoff_latitude']<=40.64))),1,0) 
#test_set['is_surge']= np.where((((test_set['hour']>=16) & (test_set['hour']<20)) & ((test_set['weekday']!=5) & (test_set['weekday']!=6))),1,0)
#test_set['is_BeforeRaise']= np.where(((test_set['year']>=2012) & (test_set['month']>=8)),1,0)

test_set_features = test_set.drop(['key','pickup_datetime'],axis=1)#'hour','day','month','weekday','lgrpickup_distance_km',

test_set_key = test_set['key']
dtest_1 = xgb.DMatrix(test_set_features)

y_pred_reg = model2.predict(dtest_1)

In [ ]:
'''test_set_features = test_set_features.drop(['year'],axis=1)#'abs_diff_longitude', 'abs_diff_latitude',
y_pred_neuralNet = neuralNetwork.predict(test_set_features)
y_pred_neuralNet = y_pred_neuralNet.reshape(9914,)
y_pred_final = (y_pred_reg+y_pred_neuralNet)/2'''

In [ ]:
#Putting everything together in a .csv file
submission = pd.DataFrame(
    {'key': test_set_key, 'fare_amount': y_pred_reg},
    columns = ['key', 'fare_amount'])
submission = submission.round({'fare_amount':2})
submission.to_csv('submission_combo.csv', index = False)
print(submission.head())
print("Submitted")

In [ ]:
submission.shape